In [1]:
! pip install pandas pymongo requests scipy scikit-learn


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from pymongo import MongoClient
import requests
from pprint import pprint
import swifter
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [3]:
books_data = pd.read_csv('datasets/BX-Books.csv',sep = ';', encoding='latin-1', error_bad_lines=False)
users_data = pd.read_csv('datasets/BX-Users.csv',sep = ';', encoding='latin-1', error_bad_lines=False)
ratings_data = pd.read_csv('datasets/BX-Book-Ratings.csv',sep = ';', encoding='latin-1', error_bad_lines=False)

Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 10
Skipping line 245933: expected 8 fields, saw 9
Skipping line 251296: expected 8 fields, saw 9
Skipping line 259941: expected 8 fields, saw 9
Skipping line 261529: expected 8 fields, saw 9

C:\Users\dhrit\AppData\Local\Temp\ipykernel_13456\1241773911.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set l

In [4]:
books_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271361 entries, 0 to 271360
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271361 non-null  object
 1   Book-Title           271361 non-null  object
 2   Book-Author          271360 non-null  object
 3   Year-Of-Publication  271361 non-null  object
 4   Publisher            271359 non-null  object
 5   Image-URL-S          271361 non-null  object
 6   Image-URL-M          271361 non-null  object
 7   Image-URL-L          271358 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [5]:
books_data['_id'] = books_data.index
books_data

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,_id
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,1
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,2
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,3
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,4
...,...,...,...,...,...,...,...,...,...
271356,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,271356
271357,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,271357
271358,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,271358
271359,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,271359


In [6]:
books_data.drop(columns=['Image-URL-S', 'Image-URL-M'], inplace = True)

In [7]:
books_data

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,_id
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,1
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,2
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,3
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,4
...,...,...,...,...,...,...,...
271356,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,271356
271357,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,271357
271358,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,271358
271359,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,271359


In [8]:
users_data

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [9]:
ratings_data

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [10]:
google_books_url = 'https://www.googleapis.com/books/v1/volumes?q=isbn:'
def get_book_info(isbn):
    response = requests.get(google_books_url+isbn)
    return response.json()

In [11]:
def update_book(book, info):
    info = info['volumeInfo']
    book['genre'] = info['categories']
    book['description'] = info['description']
    return book

def update_books_info(books):
    updated_books = []
    for i in range(len(books)):
        isbn = books[i]['ISBN']
        book_info = get_book_info(isbn)
        if book_info.get('totalItems', None):
            try:
                print(i)
                updated_books.append(update_book(books[i], book_info['items'][0]))
            except:
                print('error on', isbn)
        else:
            print(books[i])
            
    return updated_books
    

In [12]:
books_data.rename(columns = {'Book-Title':'title', 'Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher':'publisher', 'Image-URL-L':'image'}, inplace=True)
users_data.rename(columns = {'User-ID':'user_id', 'Location':'location', 'Age':'age'}, inplace=True)
ratings_data.rename(columns = {'User-ID':'user_id', 'Book-Rating':'rating'}, inplace=True)

In [13]:
ratings_data['user_id'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: user_id, Length: 105283, dtype: int64

In [14]:
x = ratings_data['user_id'].value_counts() > 200
y = x[x].index

In [15]:
ratings_data = ratings_data[ratings_data['user_id'].isin(y)]


In [16]:
ratings_data

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0
...,...,...,...
1147612,275970,3829021860,0
1147613,275970,4770019572,0
1147614,275970,896086097,0
1147615,275970,9626340762,8


In [17]:
rating_with_books = ratings_data.merge(books_data, on='ISBN')
rating_with_books.head()

,user_id,ISBN,rating,title,author,year,publisher,image,_id
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739


In [18]:
rating_with_books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 487672 entries, 0 to 487671
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    487672 non-null  int64 
 1   ISBN       487672 non-null  object
 2   rating     487672 non-null  int64 
 3   title      487672 non-null  object
 4   author     487672 non-null  object
 5   year       487672 non-null  object
 6   publisher  487670 non-null  object
 7   image      487669 non-null  object
 8   _id        487672 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 37.2+ MB


In [19]:
number_rating = rating_with_books.groupby('title')['rating'].count().reset_index()
number_rating

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1
...,...,...
160265,ï¿½?ï¿½?ber die Pflicht zum Ungehorsam gegen d...,3
160266,ï¿½?ï¿½?lpiraten.,1
160267,ï¿½?ï¿½?rger mit Produkt X. Roman.,1
160268,ï¿½?ï¿½?stlich der Berge.,1


In [20]:
number_rating.rename(columns= {'rating':'number_of_ratings'}, inplace=True)

In [21]:
final_rating = rating_with_books.merge(number_rating, on='title')
final_rating.shape

(487672, 10)

In [22]:
final_rating

,user_id,ISBN,rating,title,author,year,publisher,image,_id,number_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739,82
...,...,...,...,...,...,...,...,...,...,...
487667,275970,1892145022,0,Here Is New York,E. B. White,1999,Little Bookroom,http://images.amazon.com/images/P/1892145022.0...,53963,1
487668,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),http://images.amazon.com/images/P/1931868123.0...,54636,1
487669,275970,3411086211,10,Die Biene.,Sybil Grï¿½?Â¤fin Schï¿½?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim",http://images.amazon.com/images/P/3411086211.0...,54128,1
487670,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann,http://images.amazon.com/images/P/3829021860.0...,53958,1


In [23]:
final_rating = final_rating[final_rating['number_of_ratings'] >= 50]
final_rating.drop_duplicates(['user_id','title'], inplace=True)

C:\Users\dhrit\AppData\Local\Temp\ipykernel_13456\1085117413.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_rating.drop_duplicates(['user_id','title'], inplace=True)


In [24]:
final_rating

,user_id,ISBN,rating,title,author,year,publisher,image,_id,number_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,3739,82
...,...,...,...,...,...,...,...,...,...,...
236701,255489,0553579983,7,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,5968,50
236702,256407,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,5968,50
236703,257204,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,5968,50
236704,261829,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,5968,50


In [25]:
final_rating['title'].value_counts()

Wild Animus                                        363
The Lovely Bones: A Novel                          270
Bridget Jones's Diary                              236
The Pelican Brief                                  228
Divine Secrets of the Ya-Ya Sisterhood: A Novel    225
                                                  ... 
A Darkness More Than Night                          46
Tales of a Fourth Grade Nothing                     46
The Shining                                         46
Jacob Have I Loved                                  46
All Creatures Great and Small                       45
Name: title, Length: 742, dtype: int64

In [26]:
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values="rating")
book_pivot.fillna(0, inplace=True)
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
book_pivot.iloc[238, :]

user_id
254       9.0
2276      0.0
2766      0.0
2977      0.0
3363      0.0
         ... 
275970    0.0
277427    0.0
277478    0.0
277639    0.0
278418    0.0
Name: Harry Potter and the Goblet of Fire (Book 4), Length: 888, dtype: float64

In [28]:
books_database = final_rating.drop_duplicates(subset=['title']).drop(columns=['user_id', '_id', 'rating']).reset_index(drop=True)

In [29]:
books_database['_id'] = books_database.index

In [30]:
books_database

,ISBN,title,author,year,publisher,image,number_of_ratings,_id
0,002542730X,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82,0
1,0060930535,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133,1
2,0060934417,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,108,2
3,0061009059,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,108,3
4,006440188X,The Secret Garden,Frances Hodgson Burnett,1998,HarperTrophy,http://images.amazon.com/images/P/006440188X.0...,79,4
...,...,...,...,...,...,...,...,...
737,0515137111,Birthright,Nora Roberts,2004,Jove Books,http://images.amazon.com/images/P/0515137111.0...,52,737
738,0671867156,Pretend You Don't See Her,Mary Higgins Clark,1998,Pocket,http://images.amazon.com/images/P/0671867156.0...,105,738
739,0553287397,Dark Angel,Sally Beauman,1991,Bantam Books,http://images.amazon.com/images/P/0553287397.0...,54,739
740,0553292722,Still Waters,TAMI HOAG,1992,Bantam,http://images.amazon.com/images/P/0553292722.0...,83,740


In [31]:
books_database.to_dict('records')

[{'ISBN': '002542730X',
  'title': 'Politically Correct Bedtime Stories: Modern Tales for Our Life and Times',
  'author': 'James Finn Garner',
  'year': 1994,
  'publisher': 'John Wiley &amp; Sons Inc',
  'image': 'http://images.amazon.com/images/P/002542730X.01.LZZZZZZZ.jpg',
  'number_of_ratings': 82,
  '_id': 0},
 {'ISBN': '0060930535',
  'title': 'The Poisonwood Bible: A Novel',
  'author': 'Barbara Kingsolver',
  'year': 1999,
  'publisher': 'Perennial',
  'image': 'http://images.amazon.com/images/P/0060930535.01.LZZZZZZZ.jpg',
  'number_of_ratings': 133,
  '_id': 1},
 {'ISBN': '0060934417',
  'title': 'Bel Canto: A Novel',
  'author': 'Ann Patchett',
  'year': 2002,
  'publisher': 'Perennial',
  'image': 'http://images.amazon.com/images/P/0060934417.01.LZZZZZZZ.jpg',
  'number_of_ratings': 108,
  '_id': 2},
 {'ISBN': '0061009059',
  'title': 'One for the Money (Stephanie Plum Novels (Paperback))',
  'author': 'Janet Evanovich',
  'year': 1995,
  'publisher': 'HarperTorch',
  'im

In [1]:
import pickle
f = open('books-data.pkl', 'rb')
a = pickle.load(f)
a

[{'Unnamed: 0': 0,
  'ISBN': '002542730X',
  'title': 'Politically Correct Bedtime Stories: Modern Tales for Our Life and Times',
  'author': 'James Finn Garner',
  'year': 1994,
  'publisher': 'John Wiley &amp; Sons Inc',
  'image': 'http://images.amazon.com/images/P/002542730X.01.LZZZZZZZ.jpg',
  'number_of_ratings': 82,
  '_id': 0,
  'genre': ['Humor'],
  'description': 'A whimsical adaptation of classic fairy tales and bedtime stories removes all kinds of bias and objective language from such traditional tales as "Chicken Little," "Rapunzel," "The Three Little Pigs," "Cinderella," and many others.'},
 {'Unnamed: 0': 1,
  'ISBN': '0060930535',
  'title': 'The Poisonwood Bible: A Novel',
  'author': 'Barbara Kingsolver',
  'year': 1999,
  'publisher': 'Perennial',
  'image': 'http://images.amazon.com/images/P/0060930535.01.LZZZZZZZ.jpg',
  'number_of_ratings': 133,
  '_id': 1,
  'genre': ['Fiction'],
  'description': 'AN OPRAH BOOK CLUB SELECTION.'},
 {'Unnamed: 0': 2,
  'ISBN': '006